In [1]:
import pandas as pd

In [3]:
train = pd.read_csv("./data/open/train.csv")
test = pd.read_csv("./data/open/test.csv")

In [4]:
train

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher
...,...,...,...
15829,./train/TRAIN_15829.jpg,./upscale_train/TRAIN_15829.png,Common Myna
15830,./train/TRAIN_15830.jpg,./upscale_train/TRAIN_15830.png,Common Kingfisher
15831,./train/TRAIN_15831.jpg,./upscale_train/TRAIN_15831.png,Cattle Egret
15832,./train/TRAIN_15832.jpg,./upscale_train/TRAIN_15832.png,Ruddy Shelduck


In [6]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

conv2 = tf.keras.Sequential()
conv2.add(Conv2D(10, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
conv2.add(MaxPooling2D((2, 2)))
conv2.add(Flatten())
conv2.add(Dropout(0.5))
conv2.add(Dense(100, activation='relu'))
conv2.add(Dense(10, activation='softmax'))

In [10]:
conv2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 10)     │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1960)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1960)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │       196,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 197,210 (770.35 KB)

 Trainable params: 197,210 (770.35 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
conv2.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
from sklearn.model_selection import train_test_split

# 이미지 경로와 레이블을 추출합니다.
image_paths = train['img_path'].values
upscale_image_paths = train['upscale_img_path'].values
labels = train['label'].values

# # 학습용 데이터와 테스트용 데이터를 분할합니다.
x_train_img, x_test_img, x_train_upscale, x_test_upscale, y_train, y_test = train_test_split(
    image_paths, upscale_image_paths, labels, test_size=0.2, random_state=42
)

# (x_train_all, y_train_all), (x_test, y_test) 형식으로 데이터를 변환합니다.
x_train_all = (x_train_img, x_train_upscale)
x_test = (x_test_img, x_test_upscale)
y_train_all = y_train


In [22]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all, 
                                                  test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [2, 12667]

In [ ]:
y_train_encoded = tf.keras.utils.to_categoricaal(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)